# Conversion to ONNX
ONNX is a different format for running machine learning models. The ONNX format is much faster on CPU, sometimes 5 times as fast as PyTorch!

While the EAWSW model is designed to be small, accurate and accessible, for some people it's still too much to run...

Hosting the model as a free service for players is an option. An ONNX version of the model allows us to host the model on CPU yet have faster response times! Given that the model is made in a time with chip shortage, running on hardware I already have inside a server is efficient, scalable and cheaper.

An important note is that ONNX doesn't execute logic by itself, and you have to do that yourself, `onnx_model_manager.py` intends to deal with this for us.

In [1]:
%load_ext autoreload
%autoreload 2

from model_utils import train_model, split_data, split_branches, get_model, set_pretrained_model_dropout, get_dataset, ModelSeeder
from config import Config
import json
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random
import time
import onnx
import logging
from onnx_model_manager import OnnxModelManager
from onnxruntime.quantization import quantize_dynamic, QuantType
import os
import datasets
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from model_manager import ModelManager

In [2]:
saved_model_path = os.path.join("models", "awsw_mixed")
saved_model_onnx_path = os.path.join("models", "awsw_onnx")
if not os.path.exists(os.path.join(saved_model_path, "special_tokens_map.json")):
    print("Copying config files from huggingface (needed for conversion)... WARNING: this assumes the structure of the model isn't changed!")
    !cd $saved_model_path && git clone https://huggingface.co/EleutherAI/gpt-neo-125M
    !cp -n $saved_model_path/gpt-neo-125M/* $saved_model_path
    !rm -rf $saved_model_path/gpt-neo-125M
if not os.path.exists(os.path.join(saved_model_onnx_path, "model.onnx")):
    !python3 -m transformers.onnx --model=$saved_model_path --feature=causal-lm-with-past $saved_model_onnx_path

Using framework PyTorch: 1.10.1+cu113
Overriding 1 configuration item(s)
	- use_cache -> True
/usr/local/lib/python3.8/dist-packages/torch/onnx/utils.py:90: UserWarning: 'enable_onnx_checker' is deprecated and ignored. It will be removed in the next PyTorch release. To proceed despite ONNX checker failures, catch torch.onnx.ONNXCheckerError.
  warnings.warn("'enable_onnx_checker' is deprecated and ignored. It will be removed in "
/usr/local/lib/python3.8/dist-packages/torch/onnx/utils.py:103: UserWarning: `use_external_data_format' is deprecated and ignored. Will be removed in next PyTorch release. The code will work as it is False if models are not larger than 2GB, Otherwise set to False because of size limits imposed by Protocol Buffers.
  warnings.warn("`use_external_data_format' is deprecated and ignored. Will be removed in next "
/usr/local/lib/python3.8/dist-packages/transformers/models/gpt_neo/modeling_gpt_neo.py:559: TracerWarning: Converting a tensor to a Python boolean might 

In [3]:
def optimize_onnx():
    model_quant = os.path.join(saved_model_onnx_path, "model_quant.onnx")
    if not os.path.exists(model_quant):
        model_fp32 = os.path.join(saved_model_onnx_path, "model.onnx")
        model_opt = os.path.join(saved_model_onnx_path, "model-opt.onnx")
        quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type = QuantType.QInt8)
        #!rm $model_opt
optimize_onnx()

In [4]:
# Tell pytorch to run this model on the GPU.
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

onnx_model_manager = OnnxModelManager(os.path.join(saved_model_onnx_path, "model-opt.onnx"))
onnx_model_manager_quant = OnnxModelManager(os.path.join(saved_model_onnx_path, "model_quant.onnx"))
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')
model = AutoModelForCausalLM.from_pretrained(saved_model_path)
model.to(device)
model.eval()
model_manager = ModelManager(model=model, tokenizer=tokenizer, device=device)
print(f"Pretrained model loaded on {device_name}")

Pretrained model loaded on cuda:0


In [5]:
prompt = "In my dreams, I'm a dragon"
for i in range(10):
    print("ONNX:", onnx_model_manager.say_raw(prompt, do_sample=True))
    print("ONNX (Quantized):", onnx_model_manager_quant.say_raw(prompt, do_sample=True))
    print("PyTorch:", model_manager.say_raw(prompt, 50, 0.7))
    print('-' * 100)

ONNX: In my dreams, I'm a dragon. I've seen it myself. I can't just go back like this."<d><Fur is a dragon that's my best friend, my best caretaker, and my best bet at finding the truth. How about you find me a dragon that's not my best friend?"<d><Fur is my best friend, my best caretaker, and my best bet at finding the truth."<<|endoftext|>
ONNX (Quantized): In my dreams, I'm a dragon. I've seen it myself. I'm a dragon who can fly in the air and in the air. I've seen it myself. I'm a dragon who can fly in the air and in the air. I'm a dragon who can fly in the air and in the air. I'm a dragon who can fly in the air and in the air. I'm a dragon who can fly in the air and in the air. I'm a dragon who can fly in the air and in the air. I'm a dragon who can fly in the air and in the air. I'm a dragon who can fly in the air
PyTorch: In my dreams, I'm a dragon. I've seen it myself. It's a shadow of the human who has died and thus changed into a different form. It's a human who has died and 

# Testing

We created a few past (for context) + present prompts (player input) and see the different reactions. This way, we can test the models across different iterations.
The first test involves a old prompt to compare the pre-trained model with the one trained on AWSW. Did it manage to store it's data well? Is it able to write down things that have nothing to do with AWSW? (So we know we didn't overfit).

**This test generates boring and repetetive** replies! It's because we use no good sampling algorithm, but it does give us a indication of what the model has learned!

In [6]:
prompts = [
    ('<p><msg>c "Hey Remy!"<d><scn>park2<msg>Ry "Hey!"', "How are you?"),
    ('<p><msg>c "I was with Lorem today."<d><scn>park2<msg>Ad "Very nice."', "What do you think of Lorem?"),
    ('<p><msg>m "In Tatsu park, Adine and I sat down."', "Oh my god, Adine. What is this?"),
    ('<p><msg>m "I sat down on a chair in Anna\'s lab."', "What will we do here?"),
]

for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry "I'm fine. I'll just take a look at the apartment."<d><scn>park2<msg>Ry "Well, I guess I'll leave you to recuperate now, and maybe I'll see you next time."<|endoftext|>

[ONNX] Reply: park2<msg>Ry "I'm fine. I'll just take a look at the apartment."<d><scn>park2<msg>Ry "Well, I guess I'll leave you to recuperate now, and maybe I'll see you next time."<|endoftext|>

[ONNX Quantized] Reply: park2<msg>Ry "I'm not sure. I'll have to see if I can make it. I'll have to think about that one for a minute."<d><scn>park2<msg>Ry "I'll try."<|endoftext|>

----------
Prompt: What do you think of Lorem?
[Pytorch] Reply: park2<msg>Ad "I think he's in the middle of a heated conversation."<d><scn>park2<msg>Ad "I'm not sure, but I think it's better if we go our separate ways."<d><scn>park2<msg>Ad "I suppose that's true."<|endoftext|>

[ONNX] Reply: park2<msg>Ad "I think he's in the middle of a heated conversation."<d><scn>park2<msg>Ad "I'm not sure, but 

# Sampling test

This is gonna be interesting!

In [7]:
for (past, prompt) in prompts:
    print(f"Prompt: {prompt}")
    reply = model_manager.say(past, prompt, top_k = 50, top_p = 0.7)
    print(f"[Pytorch] Reply: {reply}\n")
    reply = onnx_model_manager.say(past, prompt, do_sample = True)
    print(f"[ONNX] Reply: {reply}\n")
    reply = onnx_model_manager_quant.say(past, prompt, do_sample = True)
    print(f"[ONNX Quantized] Reply: {reply}\n")
    print("-" * 10)

Prompt: How are you?
[Pytorch] Reply: park2<msg>Ry "It's a long story."<p><msg>c "I'm afraid so."<p><msg>c "I'm not sure what you're talking about."<d><scn>park2<msg>Ry "I'm talking about the human human."<p><msg>c "I'm talking about the human human."<p><msg>c "I

[ONNX] Reply: park2<msg>Ry "It's easy to say no to, because you're so different from other people."<d><scn>park2<msg>Ry "I know what you're talking of and I'll take that as well. So, what's the plan?"<p><msg>c "It would've been a neat change from what I was used to."<d><scn>park2<msg>Ry "I see."<p><msg>c "You're right. You don't seem to mind that I'll have to take your seat, right?"<d><sc

[ONNX Quantized] Reply: park2<msg>Ry "I'm just fine... just as I was just fine during my break."<d><scn>park2<msg>Ry "I'll see myself out of the picture for a while, I'm sure."<p><msg>c "Hey, Remy!"<d><scn>park2<msg>Ry "Oh, you know how to take your food, the food is in the fridge and the coffee are in the bowl."<d><scn>park2<msg>Ry "I'll k

# RP test
Testing out the injected roleplay actions

In [8]:
test_rps = [
    "Visit Lorem",
    "Meet with Lorem",
    "Visit Adine",
    "Fight Maverick",
    "Fight Adine",
    "Attack Adine"
]

for rp in test_rps:
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)
    
print("Lowercase test")

for rp in test_rps:
    rp = rp[0].lower() + rp[1:]
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    rp = rp.lower()
    print(f'[Pytorch] {rp} -> {model_manager.say("", rp, top_k = 50, top_p = 0.7)}')
    print(f'[ONNX] {rp} -> {onnx_model_manager.say("", rp, do_sample = True)}')
    print(f'[ONNX Quantized] {rp} -> {onnx_model_manager_quant.say("", rp, do_sample = True)}')
    print("-" * 10)

[Pytorch] Visit Lorem -> loremapt<msg>Lo "Oh, [player_name], I didn't expect you to arrive so early."<|endoftext|>
[ONNX] Visit Lorem -> loremapt<msg>Lo "Oh. I see what you're doing."<|endoftext|>
[ONNX Quantized] Visit Lorem -> loremapt<msg>Lo "Hey, Lorem. You can't just go to him and threaten him if you want to, you have to know how to react."<d><scn>loremapt<msg>Ip "I'm not so sure of how you could react, and it's up to you. You have my support, though."<d><scn>loremapt<msg>Ip "If you're so confident you're not making a scene, well, you should try out a different nosedive instead. That's what I'd prefer."<d><scn
----------
[Pytorch] Meet with Lorem -> loremapt<msg>Lo "Hey, Lorem. You're a real lifesaver, though."<|endoftext|>
[ONNX] Meet with Lorem -> loremapt<msg>Lo "Hey, Lorem."<d><scn>loremapt<msg>Lo "I thought you had experiments to run, but you don't have to worry. I'm here on my own."<d><scn>loremapt<msg>Ip "Oh."<d><scn>loremapt<msg>Ip "I'm just doing what you're supposed to d